![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# 使用watsonx, 和 LangChain 對語言模型進行一系列調用

#### 聲明

- 僅使用在watsonx可用的項目和空間。


## Notebook内容

這篇notebook包含步驟和程式碼，支援透過watsonx模型和langchain整合來示範Simple Sequential Chain。對於Python有一定的熟練度會有幫助。這篇notebook使用Pythong 3.10。


## 學習目標

這篇notebook的目標是示範串聯`google/flan-ul2`和`google/flan-t5-xxl`模型來產生一系列關於給定主題的隨機問題和該問題的答案，讓使用者熟悉LangChain框架，使用WatsonxLLM, 簡單鏈(LLMChain) 和擴充鏈(SimpleSequentialChain)。


## 内容

這篇notebook包含下列部分：

- [環境配置](#setup)
- [watsonx基礎模型](#models)
- [LangChain 串接](#watsonxllm)
- [Simple Sequential Chain實驗](#experiment)
- [總結](#summary)

<a id="setup"></a>
## 配置環境

在開始使用notebook的範例程式碼之前，需要完成下列設定任務：

-  連接一個 <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) 實例</a> instance (關於如何建立實例的信息 <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### 安裝並匯入 `datasets` 和依賴套件

In [2]:
!pip install "ibm-watson-machine-learning>=1.0.327" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install langchain | tail -n 1

### 定義 WML credentials
這個單元定義使用watsonx基礎模型推理所需的WML credentials。

**操作:** 提供IBM Cloud用戶API Key，詳情參考
[文檔](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui)。

In [1]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

### 定義 project id
基礎模型需要project id，用於提供模型呼叫上下文資訊。我們將從運行此筆記本的項目中獲取id。否則，請另外提供project id。

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## `watsonx.ai`的基礎模型

#### 列出可用的模型

所有可用的模型都顯示在ModelTypes類別下。
更多資訊可參考 [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

In [3]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT']


需要定義用於模型推理的 `model_id`:

In [4]:
model_id_1 = ModelTypes.FLAN_UL2
model_id_2 = ModelTypes.FLAN_T5_XXL

### 定義模型參數

可能需要調整模型 `parameters` 用於不同的模型和任務, 具體可參考 `GenTextParamsMetaNames` 類別.

**操作:** 詳細API可參考 [documentation](https://ibm.github.io/watson-machine-learning-sdk/).

In [5]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

### 初始化模型
用之前定義的參數初始化 `Model` 類別。

In [6]:
from ibm_watson_machine_learning.foundation_models import Model

flan_ul2_model = Model(
    model_id=model_id_1, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = Model(
    model_id=model_id_2,
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## LangChain集成

`WatsonxLLM` 封裝 watsonx.ai模型，提供模型的鏈集成。

**操作:** 更多關於`CustomLLM`可參考 [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm)


### 初始化 `WatsonxLLM` 類

In [7]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

flan_ul2_llm = flan_ul2_model.to_langchain()
flan_t5_llm = flan_t5_model.to_langchain()

**提示:** 如果需要將watsonx.ai模型與LangChain的Chain介面結合使用，請呼叫`model.to_langchain()`方法。會回傳 `WatsonxLLM`類，與LangChain CustomLLM 要求相容。

In [10]:
flan_ul2_model.to_langchain()

WatsonxLLM(model=<ibm_watson_machine_learning.foundation_models.model.Model object at 0x7f1f2dc05b70>)

使用`dict()`方法，秀出所有WatsonxLLM物件中定義的資料。

In [8]:
flan_ul2_llm.dict()

{'model_id': 'google/flan-ul2',
 'params': {'decoding_method': <DecodingMethods.SAMPLE: 'sample'>,
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '3543304f-de31-4458-8f2a-a31ceac23f79',
 'space_id': None,
 '_type': 'IBM watsonx.ai'}

<a id="experiment"></a>
## Simple Sequential Chain實驗
最簡單的sequential chain類型是 `SimpleSequentialChain`, 其中每個步驟都有一個輸入和輸出，上一個步驟的輸出會作為下一個步驟的輸入。

該實驗將包括產生一個關於任何主題的隨機問題並回答該問題。

`PromptTemplate`物件透過使用者輸入、附加非靜態資料和固定模板字串的組合來幫助產生提示。

在我們的例子中，我們想要建立兩個`PromptTemplate`對象，分別用於負責建立一個隨機問題並回答它。

In [9]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

我們希望使用「LLMChain」添加語言模型的功能.

`prompt_to_flan_ul2` 鏈規定了提示模板的格式，其任務是產生隨機問題，將字串傳遞給LLM並傳回LLM輸出。

**提示:** 如果需要將watsonx.ai模型與LangChain的Chain介面結合使用，請呼叫`model.to_langchain()`方法。會回傳 `WatsonxLLM`類，與LangChain CustomLLM 要求相容。

In [13]:
from langchain.chains import LLMChain

prompt_to_flan_ul2 = LLMChain(llm=flan_ul2_model.to_langchain(), prompt=prompt_1)

`flan_to_t5` 鏈規定了提示模版的格式，其任務是回答我們從`prompt_to_flan_ul2`鏈得到的問題, 將字串傳遞給LLM並返回LLM輸出。

In [14]:
flan_to_t5 = LLMChain(llm=flan_t5_model.to_langchain(), prompt=prompt_2)

這是完整的鏈，我們運行 `prompt_to_flan_ul2` 和 `flan_to_t5` 按序列組成的鏈.

In [ ]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_flan_ul2, flan_to_t5], verbose=True)

產生關於某個話題的隨機問題並回答。

In [16]:
qa.run('car rental')



> Entering new SimpleSequentialChain chain...
What is the most common car rental agency in the US?
alamo

> Finished chain.


'alamo'

## 產生中文話題的隨機問題並回答

In [150]:
parameters_pt1 = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.STOP_SEQUENCES: ['\n'],
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1
}

parameters_pt2 = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1
}

llama2_model_pt1 = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT, 
    params=parameters_pt1, 
    credentials=credentials,
    project_id=project_id)

llama2_model_pt2 = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT, 
    params=parameters_pt2, 
    credentials=credentials,
    project_id=project_id)

In [151]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="""產生1個與{topic}相關的問題：
    問題：
    """
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="""用1句話回答下列問題：
    問題：{question}
    答案： """
)

In [152]:
prompt_to_llama2 = LLMChain(llm=llama2_model_pt1.to_langchain(), prompt=prompt_1)
llama2_to_llama2 = LLMChain(llm=llama2_model_pt2.to_langchain(), prompt=prompt_2)

In [153]:
qa = SimpleSequentialChain(chains=[prompt_to_llama2, llama2_to_llama2], verbose=True)

In [157]:
qa.run('租車')



> Entering new SimpleSequentialChain chain...
我想租一辆车，但是我不知道哪些租车公司可以在我所在的城市提供服务，也不知道它们的价格和质量是怎样的。

可以在网上搜索“租车公司 + 你所在的城市”，或者咨询当地旅游信息中心或朋友圈，了解不同租车公司的价格和质量，并且可以预订在线或者通过电话

> Finished chain.


'可以在网上搜索“租车公司 + 你所在的城市”，或者咨询当地旅游信息中心或朋友圈，了解不同租车公司的价格和质量，并且可以预订在线或者通过电话'

<a id="summary"></a>
## 總結和下一步

你順利地完成了這篇notebook!
 
你學習如何透過custom llm WastonxLLM使用Simple Squential Chain。
 
Check out our _[Online Documentation](https://ibm.github.io/watson-machine-learning-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.